# Demo T1 processing notebook

## Choose dataset to process

In [ ]:
% Datasets available: 
% '350mT_NIST.json'
% '3T_NIST.json'
% '3T_human.json'
osfDataset = '3T_NIST.json';


## Startup qMRLab

In [ ]:
try
    cd qMRLab
    startup
    cd ..
catch
    error("qMRLab could not be started correctly.")
end

## Get configuration options



In [ ]:
datasetDetails=loadjson(osfDataset);
submission = fieldnames(datasetDetails);

dataTypeList = {};
dataSubTypeList = {};
imagePathList = {};
counter = 1;

for ii = 1:length(submission)
    datasetList = fieldnames(datasetDetails.(submission{ii}).datasets);

    for jj = 1:length(datasetList)
        OSF_linkList{counter} = datasetDetails.(submission{ii}).OSF_link;

        siteDetails=datasetDetails.(submission{ii}).datasets.(datasetList{jj});
        dataTypeList{counter} = siteDetails.dataType;
        if strcmp(siteDetails.dataType, 'Complex')
            if isfield(siteDetails, 'magnitude')
                dataSubTypeList{counter} = 'magnitude';
                imagePathList{counter} = siteDetails.magnitude.imagePath;
                counter = counter + 1;
                OSF_linkList{counter} = datasetDetails.(submission{ii}).OSF_link;
                dataTypeList{counter} = siteDetails.dataType;
                dataSubTypeList{counter} = 'phase';
                imagePathList{counter} = siteDetails.phase.imagePath;
            elseif isfield(siteDetails, 'real')
                dataSubTypeList{counter} = 'real';
                imagePathList{counter} = siteDetails.real.imagePath;
                counter = counter + 1;
                OSF_linkList{counter} = datasetDetails.(submission{ii}).OSF_link;
                dataSubTypeList{counter} = 'imaginary';
                dataTypeList{counter} = siteDetails.dataType;
                imagePathList{counter} = siteDetails.imaginary.imagePath;                
            else
                error('If datatype is complex, the side details must have either fields called either magnitude/phase or real/imaginary.')
            end
            counter = counter + 1;
        else
            dataSubTypeList{counter} = 'Magnitude';
            imagePathList{counter} = siteDetails.imagePath;
            counter = counter + 1;
        end
    end
end


## Download data

In [ ]:
downloadedList = {''};
counter = 1;

for ii = 1:length(imagePathList)

    OSF_link= OSF_linkList{ii};

    if ~strcmp(OSF_link, downloadedList)
        % Download data
        download_data(OSF_link, osfDataset);
        
        downloadedList{counter} = OSF_link;
        counter = counter + 1;
    end
    
end

## Convert YAML config files to YAML

In [ ]:
%get dataTypeList --from MATLAB
%get imagePathList --from MATLAB
%get osfDataset --from MATLAB
from convert_yaml_to_json import *

for dataType, imagePath in zip(dataTypeList, imagePathList):
    print(imagePath)
    convert_yaml_to_json(dataType, imagePath, osfDataset)


## Process entire dataset

In [ ]:
disp('Beginning dataset processing phase.')

track_complex = false;

for ii = 1:length(imagePathList)
    disp(['----- Started dataset ', num2str(ii), ' out of ', num2str(length(imagePathList)), ' -----'])

    if track_complex
        track_complex = false;
        disp('Skipping, second file of already processed complex dataset.')
        continue
    end
    dataType = dataTypeList{ii};
    imagePath = imagePathList{ii};

    if strcmp(dataType, 'Complex')
        disp('Complex dataset encountered.')
        if  strcmp(dataSubTypeList{ii}, 'magnitude')
            % Load data
            [data_mag, hdr_mag] = load_data(imagePath, osfDataset);
            ii = ii+1;
            [data_ph, hdr_ph] = load_data(imagePathList{ii}, osfDataset);
            data = magphaseToComplex(data_mag, data_ph);
            hdr = hdr_mag;
            track_complex = true;
        elseif strcmp(dataSubTypeList{ii}, 'real')
            [data_re, hdr_re] = load_data(imagePath, osfDataset);
            ii = ii+1;
            [data_im, hdr_im] = load_data(imagePathList{ii}, osfDataset);
            data = data_re +1i*data_im;
            hdr = hdr_re;
            track_complex = true;
        end
    else
        % Load data
        [data, hdr] = load_data(imagePath, osfDataset);
    end

    % Get inversion times
    TI = load_inversion_times(imagePath, osfDataset);

    % Fit data
    FitResults = fit_inversion_recovery(data, TI, dataType);

    % Save data to nifti

    [filepath,name,ext] = fileparts(imagePath);
    if strcmp(name(end-3:end),'.nii')
       [tmp1,name,tmp2] = fileparts(name);
    end
    clear tmp1, tmp2;

    t1MapPath = [osfDataset(1:end-5) '_t1map' filesep filepath];
    if strcmp(dataType, 'Complex')
        if  strcmp(dataSubTypeList{ii}, 'phase')
            t1Filename = [name(1:end-9), 'Complex_t1map.nii.gz']; % end-9 is to remove "phase"
        elseif strcmp(dataSubTypeList{ii}, 'imaginary')
            t1Filename = [name(1:end-4), 'Complex_t1map.nii.gz']; % end-4 is to remove "real"
        end
    else
        t1Filename = [name, '_t1map.nii.gz'];
    end
    
    save_to_nifti(FitResults, hdr, t1MapPath, t1Filename)
    
    save_qa_png(FitResults, t1MapPath, t1Filename)
    
    % Copy acquisition configuration files to their respective T1 results folders
    if strcmp(dataType, 'Complex')
        copy_jsonyaml_to_t1folder(filepath, name, osfDataset, dataType, dataSubTypeList{ii})
    else
        copy_jsonyaml_to_t1folder(filepath, name, osfDataset, dataType)
    end
    
    % Add link to raw data
    
    fid = fopen([t1MapPath, filesep, 'raw_data_OSF_link.txt'],'wt');
    fprintf(fid, '%s\n', OSF_linkList{ii});
    fclose(fid);
end

disp('Completed processing entire datasets!')